In [ ]:
from pathlib import Path
import sys
sys.path.append('../')
from pma_open import *


#My image
file_path = '../Dropbox Files/hel1.pma'
image_path = "../Channel Mapping/hel1_Avg_Frame/hel1_Avg_Frame.png"
image = io.imread(image_path, as_gray=True)

#Chanel image paths
CH1_img_path = "../Channel Mapping/hel1_Avg_Frame/hel1_Avg_Frame_CH1.png"
CH2_img_path = "../Channel Mapping/hel1_Avg_Frame/hel1_Avg_Frame_CH2.png"

good_peaks_1,_ = good_peak_finder(CH1_img_path)
good_peaks_2_new,_ = good_peak_finder(CH2_img_path, sigma=2, block_size=16, scaler_percent=10, boarder=10, max_rad=3)

# Move good_peaks_1 to CH2 to display full image
good_peaks_1_CH2 = shift_peaks(good_peaks_1)
good_peaks_2_CH2 = shift_peaks(good_peaks_2_new)

# Poly Mapping
CH1_peaks_10= np.array([[14, 190], [308, 10], [493, 131], [115, 168], [273, 61], [55, 59], [179, 41], [234, 157], [365, 161], [502, 43]])
CH2_peaks_10 = np.array([[18, 449], [311, 269], [492, 388], [119, 427], [276, 319], [60, 318], [183, 300], [236, 416], [364, 419], [501, 301]])

params_x_man_10, params_y_man_10 = find_polyfit_params(CH1_peaks_10, CH2_peaks_10, degree=3)
mapped_peaks_10 = apply_polyfit_params(good_peaks_1, params_x_man_10, params_y_man_10).astype(np.uint16)
poly_pair_count_tol4_10, poly_pair_arr_CH1_tol4_10, poly_pair_arr_CH2_tol4_10 = find_polyfit_pairs(mapped_peaks_10, good_peaks_1, tolerance=4)

In [ ]:
# def static_global_background_subtraction(pma_file_path, input_array, radius, y_centre_arr, x_centre_arr):
#     frames_data = read_pma(pma_file_path) 
#     all_peaks_intensity = 0
#     pixel_count = 0
#     #filling in the circle
#     for y_centre, x_centre in zip(y_centre_arr, x_centre_arr):
#         for i in range(x_centre - radius, x_centre+ radius + 1):
#             for j in range(y_centre - radius, y_centre + radius + 1):
#                 if (i - x_centre) ** 2 + (j - y_centre) ** 2 < radius ** 2:
#                     all_peaks_intensity += int(input_array[i][j][0])
#                     pixel_count += 1
    
#     # by summing the third column of the array we exclude the yellow pixels from being included!
#     total_intensity = np.sum(input_array[:, :,2])

    
#     num_of_peaks = len(y_centre_arr)
#     num_of_peak_pixels = count_circle(radius) * num_of_peaks
#     num_of_frame_pixels = input_array.shape[0] * input_array.shape[1]

#     #avg_peak_intensity gives the avg intensity of the pixels that are not within the yellow circle
#     intensity_to_remove = (total_intensity-all_peaks_intensity) // (num_of_frame_pixels-num_of_peak_pixels)
#     corrected_frames_data = []
#     for frame in frames_data: #frame is 1D
#         frame = np.clip(frame - intensity_to_remove, 0, 255).astype(np.uint8)
#         corrected_frames_data.append(frame)
#     return corrected_frames_data